In [12]:
import tiktoken
import pandas as pd


df = pd.read_csv("../data/results/test_all.csv")
encoding = tiktoken.get_encoding("cl100k_base")


input_tokens = []
output_tokens = []

for x in df.to_dict("records"):
    num_input_tokens = len(encoding.encode(x["inputs"]))
    num_output_tokens = len(encoding.encode(x["outputs"]))
    input_tokens.append(num_input_tokens)
    output_tokens.append(num_output_tokens)


input_costs = []
output_costs = []

for tokens in input_tokens:
    input_costs.append(round(tokens * 0.0005 / 1000, 4))

for tokens in output_tokens:
    output_costs.append(round(tokens *  0.0015 / 1000, 4))

print("Avg. input token: ", sum(input_tokens)/len(input_tokens))
print("Avg. output token costs: ", sum(output_tokens)/len(output_tokens))


Avg. input token:  1599.7
Avg. output token costs:  287.0


In [17]:
from generator import GeneratorFactory

system_str = """You are a full-stack expert in validating intra-technology and cross-technology configuration dependencies.
You will be presented with configuration options found in the software project 'apollo'.
Your task is to determine whether the given configuration options actually depend on each other based on value-equality.

A value-equality dependency is present if two configuration options must have identical values in order to function correctly.
Inconsistencies in these configuration values can lead to configuration errors. 
Importantly, configuration options may have equal values by accident, meaning that there is no actual dependency, but it just happens that they have equal values."""

user_str = """Information about both configuration options, including their descriptions and prior usages are stated below:
Notice that `modelVersion` contains 4.0.0. That is currently the only
supported POM version, and is always required.

    
    
    <project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
      xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 https://maven.apache.org/xsd/maven-4.0.0.xsd">
      <modelVersion>4.0.0</modelVersion>
    
      <!-- The Basics -->
      <groupId>...</groupId>
      <artifactId>...</artifactId>
      <version>...</version>
      <packaging>...</packaging>
      <dependencies>...</dependencies>
      <parent>...</parent>
      <dependencyManagement>...</dependencyManagement>
      <modules>...</modules>
      <properties>...</properties>
    
      <!-- Build Settings -->
      <build>...</build>
      <reporting>...</reporting>
    
      <!-- More Project Information -->
      <name>...</name>
      <description>...</description>
      <url>...</url>
      <inceptionYear>...</inceptionYear>
      <licenses>...</licenses>
      <organization>...</organization>
      <developers>...</developers>
      <contributors>...</contributors>
    
      <!-- Environment Settings -->
      <issueManagement>...</issueManagement>
      <ciManagement>...</ciManagement>
      <mailingLists>...</mailingLists>
      <scm>...</scm>
      <prerequisites>...</prerequisites>
      <repositories>...</repositories>
      <pluginRepositories>...</pluginRepositories>
      <distributionManagement>...</distributionManagement>
      <profiles>...</profiles>
    </project>

# The Basics

The POM contains all necessary information about a project, as well as
configurations of plugins to be used during the build process. It is the
declarative manifestation of the "who", "what", and "where", while the build
lifecycle is the "when" and "how". That is not to say that the POM cannot
affect the flow of the lifecycle - it can. For example, by configuring the
`maven-antrun-plugin`, one can embed Apache Ant tasks inside of the POM. It is
ultimately a declaration, however.

Notice that `modelVersion` contains 4.0.0. That is currently the only
supported POM version, and is always required.

    
    
    <project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
      xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 https://maven.apache.org/xsd/maven-4.0.0.xsd">
      <modelVersion>4.0.0</modelVersion>
    
      <!-- The Basics -->
      <groupId>...</groupId>
      <artifactId>...</artifactId>
      <version>...</version>
      <packaging>...</packaging>
      <dependencies>...</dependencies>
      <parent>...</parent>
      <dependencyManagement>...</dependencyManagement>
      <modules>...</modules>
      <properties>...</properties>
    
      <!-- Build Settings -->
      <build>...</build>
      <reporting>...</reporting>
    
      <!-- More Project Information -->
      <name>...</name>
      <description>...</description>
      <url>...</url>
      <inceptionYear>...</inceptionYear>
      <licenses>...</licenses>
      <organization>...</organization>
      <developers>...</developers>
      <contributors>...</contributors>
    
      <!-- Environment Settings -->
      <issueManagement>...</issueManagement>
      <ciManagement>...</ciManagement>
      <mailingLists>...</mailingLists>
      <scm>...</scm>
      <prerequisites>...</prerequisites>
      <repositories>...</repositories>
      <pluginRepositories>...</pluginRepositories>
      <distributionManagement>...</distributionManagement>
      <profiles>...</profiles>
    </project>

# The Basics

The POM contains all necessary information about a project, as well as
configurations of plugins to be used during the build process. It is the
declarative manifestation of the "who", "what", and "where", while the build
lifecycle is the "when" and "how". That is not to say that the POM cannot
affect the flow of the lifecycle - it can. For example, by configuring the
`maven-antrun-plugin`, one can embed Apache Ant tasks inside of the POM. It is
ultimately a declaration, however.

Notice that `modelVersion` contains 4.0.0. That is currently the only
supported POM version, and is always required.

    
    
    <project xmlns="http://maven.apache.org/POM/4.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
      xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 https://maven.apache.org/xsd/maven-4.0.0.xsd">
      <modelVersion>4.0.0</modelVersion>
    
      <!-- The Basics -->
      <groupId>...</groupId>
      <artifactId>...</artifactId>
      <version>...</version>
      <packaging>...</packaging>
      <dependencies>...</dependencies>
      <parent>...</parent>
      <dependencyManagement>...</dependencyManagement>
      <modules>...</modules>
      <properties>...</properties>
    
      <!-- Build Settings -->
      <build>...</build>
      <reporting>...</reporting>
    
      <!-- More Project Information -->
      <name>...</name>
      <description>...</description>
      <url>...</url>
      <inceptionYear>...</inceptionYear>
      <licenses>...</licenses>
      <organization>...</organization>
      <developers>...</developers>
      <contributors>...</contributors>
    
      <!-- Environment Settings -->
      <issueManagement>...</issueManagement>
      <ciManagement>...</ciManagement>
      <mailingLists>...</mailingLists>
      <scm>...</scm>
      <prerequisites>...</prerequisites>
      <repositories>...</repositories>
      <pluginRepositories>...</pluginRepositories>
      <distributionManagement>...</distributionManagement>
      <profiles>...</profiles>
    </project>

# The Basics

The POM contains all necessary information about a project, as well as
configurations of plugins to be used during the build process. It is the
declarative manifestation of the "who", "what", and "where", while the build
lifecycle is the "when" and "how". That is not to say that the POM cannot
affect the flow of the lifecycle - it can. For example, by configuring the
`maven-antrun-plugin`, one can embed Apache Ant tasks inside of the POM. It is
ultimately a declaration, however.
---------------------
Given the context information, perform the following task:
Carefully evaluate whether configuration option project.modelVersion of type VERSION_NUMBER with value modelVersion:4.0.0 in apollo-audit/apollo-audit-api/pom.xml of technology mavendepends on configuration option project.modelVersion of type VERSION_NUMBER with value modelVersion:4.0.0 in apollo-biz/pom.xml of technology maven or vice versa.

Respond in a JSON format as shown below:
{{
  “plan”: string, // Write down a step-by-step plan on how to solve the task given the information above.
  “rationale”: string, // Provide a concise explanation of whether and why the configuration options depend on each other due to value-equality.
  “uncertainty”: integer, // Rate your certainty of this dependency on a scale from 0 (completely uncertain) to 10 (absolutely certain), given the context, plan, and rationale.
  “isDependency”: boolean // True if a dependency exists, or False otherwise.
}}
"""


generator = GeneratorFactory().get_generator(model_name="gpt-3.5-turbo-0125", temperature=0.0)

messages = [
    {
        "role": "system",
        "content": system_str
    },
    {
        "role": "user",
        "content":  user_str

    }
]

response = generator.generate(messages=messages)

[07/01/24 15:29:10] INFO     GPT (gpt-3.5-turbo-0125) generator  ]8;id=951727;file:///home/simisimon/GitHub/cval/src/generator.py\generator.py]8;;\:]8;id=31539;file:///home/simisimon/GitHub/cval/src/generator.py#47\47]8;;\
                             initialized.                                       
[07/01/24 15:29:23] INFO     HTTP Request: POST                  ]8;id=154755;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=675945;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1026\1026]8;;\
                             https://api.openai.com/v1/chat/comp                
                             letions "HTTP/1.1 200 OK"                          


In [18]:
response

'\n                                                                                                    '